In [2]:
import pandas as pd
df1=pd.read_csv("/content/results_improved - results_improved (1).csv") #in the results_improved file I added one more col of nlp queries
import re

nlp_queries = df1[["Text"]]
gold_queries = df1[["Gold"]]
predicted_queries = df1[["Preds_ec_trb"]]


nlp_queries.to_csv("nlp_queries.csv", index=False)
gold_queries.to_csv("gold_queries.csv", index=False)
predicted_queries.to_csv("predicted_queries.csv", index=False)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
def find_comparison_sign_regex(nlp_query):
    less_than_match = re.search(r'\b(less|smaller|shorter|fewer|below)(?:\s+than)?\b', nlp_query)
    greater_than_match = re.search(r'\b(greater|larger|longer|more|above)(?:\s+than)?\b', nlp_query)
    equal_to_match = re.search(r'\bequal\s+to\b', nlp_query)

    if less_than_match:
        return '<'
    elif greater_than_match:
        return '>'
    elif equal_to_match:
        return '='
    else:
        return None



def correct_comparison_sign(predicted_query, correct_sign):
    if correct_sign:
        return re.sub(r'(<unk>)', correct_sign, predicted_query)
    else:
        return re.sub(r'<unk>', '', predicted_query)
def correct_or_remove_tags(predicted_query):
    return re.sub(r'<.*?>','',predicted_query)

for i in range(len(nlp_queries)):
    nlp_query = nlp_queries.iloc[i]["Text"]
    gold_query = gold_queries.iloc[i]["Gold"]
    predicted_query = predicted_queries.iloc[i]["Preds_ec_trb"]

    correct_sign = find_comparison_sign_regex(nlp_query)
    if correct_sign:
        corrected_query = correct_comparison_sign(predicted_query, correct_sign)
        corrected_query = correct_or_remove_tags(corrected_query)
        predicted_queries.at[i, "Corrected_Predicted_Query"] = corrected_query
        
        print(f"NLP Query: {nlp_query} | Original: {predicted_query} | Corrected: {corrected_query}")   

NLP Query: Show the names of singers that have more than one song. | Original: select t1.name from singer as t1 join song as t2 on | Corrected: select t1.name from singer as t1 join song as t2 on
NLP Query: Which district has both stores with less than 3000 products and stores with more than 10000 products? | Original: select district from shop where store_id not in (select store | Corrected: select district from shop where store_id not in (select store
NLP Query: Count the number of dogs of an age below the average. | Original: select count(*) from dogs where age <unk>(select | Corrected: select count(*) from dogs where age <(select
NLP Query: List all song names by singers above the average age. | Original: select name from singer where age > (select avg( | Corrected: select name from singer where age > (select avg(
NLP Query: What are the African countries that have a\'a0 population less than any country in Asia? | Original: select countrycode from country where population <unk>(sel

Way 2 (Using SpaCy), if in report we want to present it in complex way

In [ ]:
nlp = spacy.load("en_core_web_sm") #for parsing nlp queries , this is a predefined nlp model

def find_comparison_sign(nlp_query):
    doc = nlp(nlp_query)
    comparison_sign = None
    for token in doc:
        if token.dep_ in ("prep", "relcl") and token.head.text in ("less", "more", "equal"):
            if token.text == "than":
                if token.head.text == "less":
                    comparison_sign = "<"
                elif token.head.text == "more":
                    comparison_sign = ">"
            elif token.text == "to":
                if token.head.text == "equal":
                    comparison_sign = "="
            break
    return comparison_sign


  